In [1]:
import sys
import os
import json
import xml.etree.ElementTree as ET

In [2]:
START_BOUNDING_BOX_ID = 1

In [3]:
PRE_DEFINE_CATEGORIES = {"person":1}

In [4]:
def get(root, name):
    vars = root.findall(name)
    return vars

In [5]:
def get_and_check(root, name, length):
    vars = root.findall(name)
    if len(vars) == 0:
        raise NotImplementedError('Can not find %s in %s.'%(name, root.tag))
    if length > 0 and len(vars) != length:
        raise NotImplementedError('The size of %s is supposed to be %d, but is %d.'%(name, length, len(vars)))
    if length == 1:
        vars = vars[0]
    return vars

In [6]:
def get_filename_as_int(filename):
    try:
        filename = os.path.splitext(filename)[0]
        return int(filename)
    except:
        raise NotImplementedError('Filename %s is supposed to be an integer.'%(filename))
 

In [7]:
def convert(xml_dir, json_file):
    xmlFiles = os.listdir(xml_dir)
    
    json_dict = {"images":[], "type": "instances", "annotations": [],
                 "categories": []}
    categories = PRE_DEFINE_CATEGORIES
    bnd_id = START_BOUNDING_BOX_ID
    num = 0
    for line in xmlFiles:
#         print("Processing %s"%(line))
        num +=1
        if num%50==0:
            print("processing ",num,"; file ",line)
        #print("processing ",num,"; file ",line)
        xml_f = os.path.join(xml_dir, line)
        tree = ET.parse(xml_f)
        root = tree.getroot()
        ## The filename must be a number
        filename = line[:-4]
        image_id = get_filename_as_int(filename)
        size = get_and_check(root, 'size', 1)
        width = int(get_and_check(size, 'width', 1).text)
        height = int(get_and_check(size, 'height', 1).text)
        # image = {'file_name': filename, 'height': height, 'width': width,
        #          'id':image_id}
        image = {'file_name': (filename+'.jpg'), 'height': height, 'width': width,
                 'id':image_id}
        json_dict['images'].append(image)
        ## Cruuently we do not support segmentation
        #  segmented = get_and_check(root, 'segmented', 1).text
        #  assert segmented == '0'
        for obj in get(root, 'object'):
            category = get_and_check(obj, 'name', 1).text
            if category == 'person':
                print(category,',',line)
            if category not in categories:
                print(category,',',line)
                new_id = len(categories)
                categories[category] = new_id
            category_id = categories[category]
            bndbox = get_and_check(obj, 'bndbox', 1)
            xmin = int(get_and_check(bndbox, 'xmin', 1).text) - 1
            ymin = int(get_and_check(bndbox, 'ymin', 1).text) - 1
            xmax = int(get_and_check(bndbox, 'xmax', 1).text)
            ymax = int(get_and_check(bndbox, 'ymax', 1).text)
            #print(xmin,xmax,ymin,ymax)
            assert(xmax > xmin)
            assert(ymax > ymin)
            o_width = abs(xmax - xmin)
            o_height = abs(ymax - ymin)
            ann = {'area': o_width*o_height, 'iscrowd': 0, 'image_id':
                   image_id, 'bbox':[xmin, ymin, o_width, o_height],
                   'category_id': category_id, 'id': bnd_id, 'ignore': 0,
                   'segmentation': []}
            json_dict['annotations'].append(ann)
            bnd_id = bnd_id + 1
 
    for cate, cid in categories.items():
        cat = {'supercategory': 'none', 'id': cid, 'name': cate}
        json_dict['categories'].append(cat)
    json_fp = open(json_file, 'w')
    json_str = json.dumps(json_dict)
    json_fp.write(json_str)
    json_fp.close()

In [9]:
base_dir = "./swim_voc/"
folder_list= ["train","val","test"]


In [11]:
for i in range(3):
    folderName = folder_list[i]
    xml_dir = base_dir + folderName + "/Annotations/"
    json_dir = base_dir + folderName + "/instances_" + folderName + ".json"
    print("deal: ",folderName)
    print("xml dir: ",xml_dir)
    print("json file: ",json_dir)
    convert(xml_dir,json_dir)

deal:  train
xml dir:  ./swim_voc/train/Annotations/
json file:  ./swim_voc/train/instances_train.json
person , 00000875.xml
person , 00000875.xml
person , 00000875.xml
person , 00001297.xml
person , 00001201.xml
person , 00001201.xml
person , 00001201.xml
person , 00001201.xml
person , 00001201.xml
person , 00001426.xml
person , 00001426.xml
person , 00001426.xml
person , 00000359.xml
person , 00000359.xml
person , 00000290.xml
person , 00000290.xml
person , 00002033.xml
person , 00002033.xml
person , 00000403.xml
person , 00001168.xml
person , 00001314.xml
person , 00001314.xml
person , 00001314.xml
person , 00001314.xml
person , 00000353.xml
person , 00001462.xml
person , 00001462.xml
person , 00000306.xml
person , 00001964.xml
person , 00001964.xml
person , 00000177.xml
person , 00000163.xml
person , 00001333.xml
person , 00001210.xml
person , 00001043.xml
person , 00002147.xml
person , 00002147.xml
person , 00000648.xml
person , 00000648.xml
person , 00001900.xml
person , 00000294

person , 00001350.xml
person , 00001350.xml
person , 00001350.xml
person , 00000513.xml
person , 00001915.xml
person , 00001165.xml
person , 00001165.xml
person , 00001451.xml
person , 00001252.xml
person , 00001252.xml
person , 00001252.xml
person , 00001094.xml
person , 00000127.xml
person , 00000588.xml
person , 00000588.xml
person , 00000588.xml
person , 00001925.xml
person , 00000451.xml
person , 00001625.xml
person , 00000316.xml
person , 00002064.xml
person , 00002064.xml
person , 00001270.xml
person , 00001270.xml
person , 00001270.xml
person , 00000935.xml
person , 00000935.xml
person , 00000935.xml
person , 00000147.xml
person , 00000147.xml
person , 00000147.xml
person , 00000147.xml
person , 00000735.xml
person , 00000735.xml
person , 00000735.xml
person , 00000344.xml
person , 00000345.xml
person , 00000680.xml
person , 00000680.xml
person , 00001214.xml
person , 00001013.xml
person , 00000237.xml
person , 00000237.xml
person , 00000237.xml
person , 00000887.xml
person , 0

person , 00000292.xml
person , 00000927.xml
person , 00001802.xml
person , 00000716.xml
person , 00000716.xml
person , 00000716.xml
person , 00000645.xml
person , 00002003.xml
person , 00001955.xml
person , 00001955.xml
person , 00001955.xml
person , 00001770.xml
person , 00001279.xml
person , 00001279.xml
person , 00000154.xml
person , 00000154.xml
person , 00000154.xml
person , 00001222.xml
person , 00001222.xml
person , 00000706.xml
person , 00000706.xml
person , 00002018.xml
person , 00002018.xml
person , 00000146.xml
person , 00001507.xml
person , 00001507.xml
person , 00001196.xml
person , 00001196.xml
person , 00001196.xml
person , 00002161.xml
person , 00000037.xml
person , 00000037.xml
person , 00001289.xml
person , 00001289.xml
person , 00001289.xml
person , 00001046.xml
processing  1550 ; file  00001614.xml
person , 00001614.xml
person , 00000466.xml
person , 00000466.xml
person , 00001603.xml
person , 00001603.xml
person , 00000381.xml
person , 00001962.xml
person , 0000196

In [ ]:
annotations["info"] = {
        "description": "customer dataset format convert to COCO format",
        "url": "",
        "version": "1.0",
        "year": 2019,
        "contributor": "oliver.xuao",
        "date_created": "2019/12/6"
    }
annotations["licenses"] = [{
        "url": "",
        "id": 1,
        "name": "HKUIT License 1.0"
    }]

In [ ]:
annotations["categories"] = []

In [ ]:
for cls, clsname in enumerate(__CLASS__):
    if clsname == '__background__':
            continue
        annotations["categories"].append(
            {
                "supercategory": "object",
                "id": cls,
                "name": clsname
            }
        )
        for catdict in annotations["categories"]:
            if "person" == catdict["name"] and args.have_points:
                catdict["keypoints"] = ["top_left", "top_right", "bottom_right", "bottom_left"]
                catdict["skeleton"] = [[]]
